Rohan Bhatt - MSML604 - HW8

In [ ]:
import numpy as np 
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# initial given problem data
P = np.array([[5, 4, 6],
              [4, 9,15], 
              [6,15,35]], dtype=float)
# linear term in the cost
q = np.array([5, 7, 6], dtype=float)

# rows of Ax ≤ b  (2 inequalities)
A = np.array([[4.0, 0.9, 2.0], 
              [1.3, 0.6, 6.0]])
# right-hand side vector for Ax ≤ b
b = np.array([1.2, 0.8], dtype=float)

#original quadratic objective function f0(x)
def f0(x):
    #returns the scalar obj function 1/2 x^T P x + q^T x
    return 0.5 * x @ P @ x + q @ x 
#log barrier augmented objective 
def barrier(x,t):
    g = np.concatenate([A @ x - b, x-1, -x-1]) #ax - b, xi-1 from xi <= 1, -xi-1 from xi >= -1
    if (g >= 0).any(): #if any inequality is not strict
        return np.inf #return +infinity so its rejected 
    barrier = -np.sum(np.log(-g)) #log barrier term (sum(−log(−g_i(x))))
    return t * f0(x) + barrier #return t times the objective function plus the barrier term

#single equality constraint 1^Tx = 1
equality_constraints = {
    #SLSQP (Sequential Least Squares Quadratic Programming) equality specification
    'type': 'eq', 
    #returs 0 when 1^Tx = 1
    'fun': lambda x: np.sum(x) - 1
}
#strictly feasible starting point x0
x0 = np.array([.05, .90, .05]) #hand picked interior point
#checking Ax < b, -1 < xi < 1, and 1^Tx = 1
assert (A @ x0 < b).all() #, "x0 is not feasible for Ax < b"
assert (np.abs(x0) < 1).all()#, "x0 is not feasible for -1 < xi < 1"
assert np.isclose(np.sum(x0), 1)#, "x0 is not feasible for 1^Tx = 1"
#central path iterations (barrier parameters t)
t_values = [1, 5, 10, 20] #sequence of t values (barier params)
f_history = [] #to store the function values f0(x*(t_k))

for t in t_values: #looping over every t_k:
    solution = minimize(
        #objective function
        barrier, 
        #initial guess
        x0, 
        #passing t into barrier obj
        args=(t,),
        #method to use
        method='SLSQP',
        #equality constraints
        constraints=[equality_constraints],
        #setting tight tolerance with no console output
        options={'disp': False, 'ftol': 1e-12}
    )
    if not solution.success:
        raise RuntimeError(f"Optimization failed: {solution.message}")
    x0 = solution.x #starting next loop with minimiser
    f_history.append(f0(x0)) #recording true objective at x*(t_k)





AssertionError: 

In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# ------------ problem data -------------
P = np.array([[5, 4, 6],
              [4, 9,15],
              [6,15,35]], dtype=float)
q = np.array([5., 7., 6.])

A = np.array([[4.0, 0.9, 2.0],
              [1.3, 0.6, 6.0]])
b = np.array([1.2, 0.8])

m      = A.shape[0] + 2*3          # total number of inequalities
μ      = 10                        # t multiplier
tol    = 1e-8                      # stop when m/t < tol
f_hist = []                        # f0(x*(t_k))
t      = 1.0                       # start with t1 = 1

# ---------- helpers ----------
def f0(x):
    return 0.5*x @ P @ x + q @ x

def make_feasible():
    """Randomly generate a strictly-feasible interior point."""
    rng = np.random.default_rng(0)
    while True:
        # sample two coordinates; enforce 1ᵀx = 1 for the third
        x12 = rng.uniform(-0.8, 0.8, 2)
        x3  = 1. - x12.sum()
        if abs(x3) >= 0.99:                # keep interior to [-1,1]
            continue
        x = np.append(x12, x3)
        if (A @ x < b - 1e-6).all() and (np.abs(x) < 1.0).all():
            return x

x0 = make_feasible()                # strictly feasible start

def barrier_obj(x, t):
    g = np.concatenate((A @ x - b, x - 1, -x - 1))   # g_i(x) < 0
    if (g >= 0).any():
        return 1e20                                  # large penalty
    return t * f0(x) - np.sum(np.log(-g))

eq_constraint = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

# ---------- central-path iterations ----------
while m / t > tol:
    res = minimize(barrier_obj,
                   x0,
                   args=(t,),
                   method='SLSQP',
                   constraints=[eq_constraint],
                   options={'ftol': 1e-12, 'disp': False})
    if not res.success:
        raise RuntimeError(res.message)

    x0 = res.x                # warm-start next barrier sub-problem
    f_hist.append(f0(x0))
    t *= μ                    # increase barrier parameter

# ---------- plot ----------
plt.figure(figsize=(6,4))
plt.semilogy(range(1, len(f_hist)+1), f_hist, marker='o')
plt.xlabel(r'iteration $k$')
plt.ylabel(r'$f_0(x^*(t_k))$')
plt.title('Central–path objective values')
plt.grid(True, which='both', ls=':')
plt.tight_layout()
plt.show()
